In [2]:
import openai
from dotenv import load_dotenv
# from scipy.spatial.distance import cosine
import os
from openai import OpenAI
import pickle
import pandas as pd
import re

C:\Users\jsy11\AppData\Local\Temp\ipykernel_3852\1520460610.py:7: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI()

In [4]:
# 피클 파일에서 데이터 로드
def load_book_data(file_path):
    with open(file_path, "rb") as file:
        book_data = pickle.load(file)
    return book_data

In [5]:
df = pd.read_pickle("../../processed/test4.pkl")

In [6]:
df

,Page,Tokens,n_tokens,embedding
1,2,"['About', 'This', 'eBook', 'ePUB', 'is', 'an',...",618,"[0.0027860531117767096, 0.008122052997350693, ..."
2,3,"['Security', 'in', 'Computing', 'FIFTH', 'EDIT...",130,"[0.014323357492685318, -0.003350171959027648, ..."
3,4,"['Many', 'of', 'the', 'designations', 'used', ...",1105,"[-0.00623887125402689, -0.011925099417567253, ..."
4,5,"['Executive', 'Editor', 'Bernard', 'Goodwin', ...",108,"[-0.012079219333827496, 0.002936786040663719, ..."
5,6,"['To', 'Willis', 'Ware', 'a', 'hero', 'of', 'c...",31,"[-0.00888099242001772, -0.012829650193452835, ..."
...,...,...,...,...
1005,1006,"['Big', 'Data', 'Conclusion', 'Exercises', 'Cl...",421,"[-0.013197178021073341, -0.0050868378020823, 0..."
1006,1007,"['PrivacyPreserving', 'Data', 'Mining', 'Priva...",497,"[-0.01291818916797638, -0.014100742526352406, ..."
1007,1008,"['Human', 'Vandals', 'Interception', 'of', 'Se...",585,"[-0.013505895622074604, -0.02323957160115242, ..."
1008,1009,"['Situation', 'IX', 'True', 'Representation', ...",417,"[0.018290884792804718, -0.005117486231029034, ..."


In [33]:
QnA={'Q':[],
     'A':[]}

In [7]:
def extract_q_and_a(question):
    # Split the question into lines
    lines = question.split('\n')
    
    # Initialize variables for Q and A
    q_part = ""
    a_part = ""
    
    # Iterate through lines to find Q: or 질문: and A: or 답변:
    for line in lines:
        if line.startswith("Q:") or line.startswith("질문:"):
            q_part = line.strip()
        elif line.startswith("A:") or line.startswith("답변:"):
            a_part = line.strip()
    
        # Replace 질문: with Q: only when 질문: is present
        if "질문:" in q_part:
            q_part = q_part.replace("질문:", "Q:")

        # Replace 답변: with A: only when 답변: is present
        if "답변:" in a_part:
            a_part = a_part.replace("답변:", "A:")

        # Make sure Q: is always at the beginning
        if not q_part.startswith("Q:"):
            q_part = f"Q: {q_part}"
    return q_part, a_part

In [8]:
def generate_quiz_question(context, model="gpt-3.5-turbo"):
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[
                {
                    "role": "system",
                    "content": "You have to make a quiz about computer security, and you have to make a question that can be answered in a sentence rather than a four-point first answer. And there should be no problems using pictures and tables. Q: A:",
                },
                {
                    "role": "user",
                    "content": f"Context: {context}\n\n---\n\n, 한국어로 대답해줘.",
                },
            ],
            temperature=0,
        )
        generated_question = response.choices[0].message.content

        # Extract Q: or 질문: and A: or 답변: parts
        q_part, a_part = extract_q_and_a(generated_question)

        # Return Q and A as a dictionary to ensure order
        return {"Q": q_part, "A": a_part}
    except Exception as e:
        print("Error occurred:", e)
        return {"Q": "I don't know", "A": "I don't know"}


In [9]:
# 피클 파일에서 로드한 데이터에서 랜덤한 문장을 선택하여 퀴즈 생성
def generate_random_quiz(df):
    # context = df.sample()["Text"].values[0]  # 예시로 랜덤하게 선택
    context = df.sample()["Tokens"].values[0]  # 예시로 랜덤하게 선택
    question = generate_quiz_question(context)
    return context, question

In [39]:
context, question = generate_random_quiz(df)

In [40]:
question['Q']

'Q: 보안 커널의 장점은 무엇인가요?'

In [41]:
question['A']

'A: 보안 커널의 장점은 보안 기능을 격리시켜 보호하고, 추적 가능하며, 변경 가능하며, 간결하며, 검증 가능하다는 것입니다.'

In [42]:
QnA['Q'].append(question['Q'])
QnA['A'].append(question['A'])

In [43]:
QnA

{'Q': ['Q: DES 알고리즘은 어떤 종류의 연산을 사용하여 구현될 수 있나요?', 'Q: 보안 커널의 장점은 무엇인가요?'],
 'A': ['A: DES 알고리즘은 테이블 조회와 논리 연산(AND, OR, XOR)을 사용하여 구현될 수 있습니다.',
  'A: 보안 커널의 장점은 보안 기능을 격리시켜 보호하고, 추적 가능하며, 변경 가능하며, 간결하며, 검증 가능하다는 것입니다.']}

In [34]:
len(QnA['Q'])

0

In [45]:
QnA['Q'][0]

'Q: DES 알고리즘은 어떤 종류의 연산을 사용하여 구현될 수 있나요?'